In [12]:
# This code should be run in console room computer to display the feedback morphings
from __future__ import print_function, division
import os
if 'watts' in os.getcwd():
    main_dir = "/home/watts/Desktop/ntblab/kailong/rtSynth_rt/"
else:
    main_dir="/Users/kailong/Desktop/rtEnv/rtSynth_rt/"
import sys
sys.path.append(main_dir)
sys.path.append(main_dir+"expScripts/feedback/")
from psychopy import visual, event, core, logging, gui, data, monitors
from psychopy.hardware.emulator import launchScan, SyncGenerator
from PIL import Image
import string
import fmrisim as sim
import numpy as np
import pandas as pd
import pylink   
from tqdm import tqdm
import time
import re
import logging
import threading
import argparse
alpha = string.ascii_uppercase
from rtCommon.subjectInterface import SubjectInterface
from rtCommon.wsRemoteService import WsRemoteService, parseConnectionArgs
from rtCommon.utils import installLoggers
from rtCommon.cfg_loading import mkdir,cfg_loading



class SubjectService:
    def __init__(self, args, webSocketChannelName='wsSubject'):
        """
        Uses the WsRemoteService framework to parse connection-related args and establish
        a connection to a remote projectServer. Instantiates a local version of
        SubjectInterface to handle client requests coming from the projectServer connection.
        Args:
            args: Argparse args related to connecting to the remote server. These include
                "-s <server>", "-u <username>", "-p <password>", "--test",
                "-i <retry-connection-interval>"
            webSocketChannelName: The websocket url extension used to connecy and communicate
                to the remote projectServer, 'wsSubject' will connect to 'ws://server:port/wsSubject'
        """
        self.subjectInterface = SubjectInterface(subjectRemote=False)
        self.wsRemoteService = WsRemoteService(args, webSocketChannelName)
        self.wsRemoteService.addHandlerClass(SubjectInterface, self.subjectInterface)

    def runDetached(self):
        """Starts the receiver in it's own thread."""
        self.recvThread = threading.Thread(name='recvThread',
                                           target=self.wsRemoteService.runForever)
        self.recvThread.setDaemon(True)
        self.recvThread.start()


argParser = argparse.ArgumentParser()

argParser.add_argument('-c', '--config', action="store", dest="config", default='sub001.ses2.toml', type=str, help='experiment file (.json or .toml)')
argParser.add_argument('-r', '--run', action="store", dest="run", default='1', type=str, help='current run')
# argParser.add_argument('-e', '--sess', action="store", dest="sess", default='1', type=str, help='current session')
argParser.add_argument('-s', action="store", dest="server", default="localhost:7777",
                    help="Server Address with Port [server:port]")

argParser.add_argument('-i', action="store", dest="interval", type=int, default=5,
                    help="Retry connection interval (seconds)")
argParser.add_argument('-u', '--username', action="store", dest="username", default='kp578',
                    help="rtcloud website username")
argParser.add_argument('-p', '--password', action="store", dest="password", default='kp578',
                    help="rtcloud website password")
argParser.add_argument('--test', default=False, action='store_true',
                    help='Use unsecure non-encrypted connection')
argParser.add_argument('--trying', default=False, action='store_true',
                    help='Use unsecure non-encrypted connection')
args = argParser.parse_args("")

args.trying=True
if args.trying:
    scanmode = 'Test'  # 'Scan' or 'Test' or None
    screenmode = False  # fullscr True or False
    monitor_name = "testMonitor" #"testMonitor"
    prange=20
else:
    scanmode = 'Scan'  # 'Scan' or 'Test' or None
    screenmode = True  # fullscr True or False
    monitor_name = "scanner"
    prange=20

if not re.match(r'.*:\d+', args.server):
    print("Error: Expecting server address in the form <servername:port>")
    argParser.print_help()
    sys.exit()

# Check if the ssl certificate is valid for this server address
from rtCommon.projectUtils import login, certFile, checkSSLCertAltName, makeSSLCertFile
addr, _ = args.server.split(':')
if checkSSLCertAltName(certFile, addr) is False:
    # Addr not listed in sslCert, recreate ssl Cert
    makeSSLCertFile(addr)


cfg = cfg_loading(args.config)
sub = cfg.subjectName
run = int(args.run)  # 1
sess = int(cfg.session)

cfg.feedback_expScripts_dir = f"{cfg.projectDir}expScripts/feedback/"

gui = True if screenmode == False else False
scnWidth, scnHeight = monitors.Monitor(monitor_name).getSizePix()
frameTolerance = 0.001  # how close to onset before 'same' frame
TRduration=int(cfg.TR)

# mywin = visual.Window(
    # size=[1280, 800], fullscr=screenmode, screen=0,
    # winType='pyglet', allowGUI=False, allowStencil=False,
    # monitor=monitor_name, color=[0,0,0], colorSpace='rgb', #color=[0,0,0]
    # blendMode='avg', useFBO=True,
    # units='height')

mywin = visual.Window(
    size=[scnWidth - 100, scnHeight - 100], fullscr=screenmode, screen=1,
    winType='pyglet', allowGUI=False, allowStencil=False,
    monitor=monitor_name, color=[0,0,0], colorSpace='rgb', #color=[0,0,0]
    blendMode='avg', useFBO=True,
    units='height')

# similation specific
step=3 #in simulation, how quickly the morph changes ramp up. Note this is only for simulation, has nothing to do with real experiment

# trial_list designing parameters
parameterRange=np.arange(1,prange) #for saving time for now. np.arange(1,20) #define the range for possible parameters for preloading images. Preloading images is to make the morphing smooth during feedback
tune=4 # this parameter controls how much to morph (how strong the morphing is) (used in preloading function), tune can range from (1,6.15] when paremeterrange is np.arange(1,20)
TrialNumber=cfg.TrialNumber # how many trials are required #test trial ,each trial is 14s, 10 trials are 140s.

## - design the trial list: the sequence of the different types of components: 
## - e.g: ITI + waiting for fMRI signal + feedback (receive model output from feedbackReceiver.py)
trial_list = pd.DataFrame(columns=['Trial','time','TR','state','newWobble'])
curTime=0
curTR=0
state=''
trial_list.append({'Trial':None,
                    'time':None,
                    'TR':None,
                    'state':None,
                    'newWobble':None},
                    ignore_index=True)

for currTrial in range(1,1+TrialNumber):

    # ITI
    for i in range(6): # should be 6TR=12s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'ITI',
                                    'newWobble':0},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1

    # waiting for metric calculation
    for i in range(3): # should be 3TR=6s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'waiting',
                                    'newWobble':0},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1
    
    # feedback trial: try minimize the whobbling
    for i in range(5): #5TR=10s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'feedback',
                                    'newWobble':1},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1

# end ITI
for i in range(6): # should be 6TR=12s
    trial_list=trial_list.append({'Trial':currTrial,
                                'time':curTime,
                                'TR':curTR,
                                'state':'ITI',
                                'newWobble':0},
                                ignore_index=True)
    curTime=curTime+TRduration
    curTR=curTR+1



# parameters = np.arange(1,step*(sum((trial_list['newWobble']==1)*1)),step) #[1,2,3,4,5,6,7,8]

print('total trial number=',TrialNumber)
# print('neighboring morph difference=',tune)
print('preloaded parameter range=',parameterRange)
# print('used parameters=',parameters)



def sample(L,num=10):
    # This functional uniformly sample the list to be num points
    # e.g, if L is 0-99, num is 10, newList would be [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]
    # e.g, if L is 0-95, num is 10, newList would be [8, 18, 27, 37, 47, 56, 66, 75, 85, 95]
    # e.g, if L is 0-5, num is 10, newList would be [0, 0, 0, 1, 2, 2, 3, 3, 4, 5]
    sampleStep=len(L)/num 
    newList=[]
    for i in range(1,num):
        newList.append(L[int(i*sampleStep-1)])
    newList.append(L[-1])
    return newList


# # preload image list for parameter from 1 to 19.
# def preloadimages(parameterRange=np.arange(1,20),tune=1):
#     '''
#     purpose:
#         preload images into image object sequences corrresponding too each parameter
#         each parameter corresponds to 40 image objects
#     steps:

#     '''
#     tune=tune-1
#     start = time.time()
#     imageLists={}
#     numberOfUpdates=16 # corresponds to 66 updates    
#     last_image=''
#     for currParameter in tqdm(parameterRange): #49
#         images=[]
#         print('maximum morph=',round((tune*currParameter*numberOfUpdates+2)/numberOfUpdates+1))
#         for axis in ['bedTable', 'benchBed']:
#             tmp_images=[]
#             for currImg in range(1,int(round(tune*currParameter*numberOfUpdates+2)),int((currParameter*numberOfUpdates+2)/numberOfUpdates)):
#                 currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
#                 if currMorph<1 or currMorph>99:
#                     raise Exception('morphing outside limit')
#                 curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#                 if curr_image!=last_image:
#                     # currImage=visual.ImageStim(win=mywin,
#                     #                             name='image',
#                     #                             image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
#                     #                             ori=0, pos=(0, 0), size=(0.5, 0.5),
#                     #                             color=[1,1,1], colorSpace='rgb', opacity=1,
#                     #                             flipHoriz=False, flipVert=False,
#                     #                             texRes=128, interpolate=True, depth=-4.0)
#                     currImage = cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#                 tmp_images.append(currImage)
#                 last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#             images=images+sample(tmp_images)
#             tmp_images=[]
#             for currImg in reversed(range(1,int(round(tune*currParameter*numberOfUpdates+1)),int((currParameter*numberOfUpdates+2)/numberOfUpdates))):
#                 currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
#                 curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#                 if curr_image!=last_image:
#                     # currImage=visual.ImageStim(win=mywin,
#                     #                             name='image',
#                     #                             image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
#                     #                             ori=0, pos=(0, 0), size=(0.5, 0.5),
#                     #                             color=[1,1,1], colorSpace='rgb', opacity=1,
#                     #                             flipHoriz=False, flipVert=False,
#                     #                             texRes=128, interpolate=True, depth=-4.0)
#                     currImage = image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#                 tmp_images.append(currImage)
#                 last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
#             images=images+sample(tmp_images)
#         imageLists.update( {currParameter : images} )
#     end = time.time()
#     print("preload image duration=", end - start)
#     return imageLists
# preload image list for parameter from 1 to 19.
def preloadimages(parameterRange=np.arange(1,20),tune=1):
    '''
    purpose:
        preload images into image object sequences corrresponding too each parameter
        each parameter corresponds to 40 image objects
    steps:

    '''
    tune=tune-1
    start = time.time()
    imageLists={}
    numberOfUpdates=16 # corresponds to 66 updates    
    last_image=''
    for currParameter in tqdm(parameterRange): #49
        images=[]
        print('maximum morph=',round((tune*currParameter*numberOfUpdates+2)/numberOfUpdates+1))
        for axis in ['bedTable', 'benchBed']:
            tmp_images=[]
            for currImg in range(1,int(round(tune*currParameter*numberOfUpdates+2)),int((currParameter*numberOfUpdates+2)/numberOfUpdates)):
                currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
                if currMorph<1 or currMorph>99:
                    raise Exception('morphing outside limit')
                curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                if curr_image!=last_image:
                    # currImage=visual.ImageStim(win=mywin,
                    #                             name='image',
                    #                             image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
                    #                             ori=0, pos=(0, 0), size=(0.5, 0.5),
                    #                             color=[1,1,1], colorSpace='rgb', opacity=1,
                    #                             flipHoriz=False, flipVert=False,
                    #                             texRes=128, interpolate=True, depth=-4.0)
                    currImage = cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                tmp_images.append(currImage)
                last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
            images=images+sample(tmp_images)
            tmp_images=[]
            for currImg in reversed(range(1,int(round(tune*currParameter*numberOfUpdates+1)),int((currParameter*numberOfUpdates+2)/numberOfUpdates))):
                currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
                curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                if curr_image!=last_image:
                    # currImage=visual.ImageStim(win=mywin,
                    #                             name='image',
                    #                             image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
                    #                             ori=0, pos=(0, 0), size=(0.5, 0.5),
                    #                             color=[1,1,1], colorSpace='rgb', opacity=1,
                    #                             flipHoriz=False, flipVert=False,
                    #                             texRes=128, interpolate=True, depth=-4.0)
                    currImage = cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                tmp_images.append(currImage)
                last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
            images=images+sample(tmp_images)
        imageLists.update( {currParameter : images} )
    end = time.time()
    print("preload image duration=", end - start)
    return imageLists


_=time.time()
imageLists=preloadimages(parameterRange=np.arange(1,33),tune=tune)

toml filename=/Users/kailong/Desktop/rtEnv/rtSynth_rt/projects/rtSynth_rt/conf/sub001.ses2.toml


100%|██████████| 32/32 [00:00<00:00, 1232.18it/s]

total trial number= 12
preloaded parameter range= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
maximum morph= 4.0
maximum morph= 7.0
maximum morph= 10.0
maximum morph= 13.0
maximum morph= 16.0
maximum morph= 19.0
maximum morph= 22.0
maximum morph= 25.0
maximum morph= 28.0
maximum morph= 31.0
maximum morph= 34.0
maximum morph= 37.0
maximum morph= 40.0
maximum morph= 43.0
maximum morph= 46.0
maximum morph= 49.0
maximum morph= 52.0
maximum morph= 55.0
maximum morph= 58.0
maximum morph= 61.0
maximum morph= 64.0
maximum morph= 67.0
maximum morph= 70.0
maximum morph= 73.0
maximum morph= 76.0
maximum morph= 79.0
maximum morph= 82.0
maximum morph= 85.0
maximum morph= 88.0
maximum morph= 91.0
maximum morph= 94.0
maximum morph= 97.0
preload image duration= 0.029369115829467773


100%|██████████| 32/32 [00:00<00:00, 703.54it/s]

maximum morph= 4.0
maximum morph= 7.0
maximum morph= 10.0
maximum morph= 13.0
maximum morph= 16.0
maximum morph= 19.0
maximum morph= 22.0
maximum morph= 25.0
maximum morph= 28.0
maximum morph= 31.0
maximum morph= 34.0
maximum morph= 37.0
maximum morph= 40.0
maximum morph= 43.0
maximum morph= 46.0
maximum morph= 49.0
maximum morph= 52.0
maximum morph= 55.0
maximum morph= 58.0
maximum morph= 61.0
maximum morph= 64.0
maximum morph= 67.0
maximum morph= 70.0
maximum morph= 73.0
maximum morph= 76.0
maximum morph= 79.0
maximum morph= 82.0
maximum morph= 85.0
maximum morph= 88.0
maximum morph= 91.0
maximum morph= 94.0
maximum morph= 97.0
preload image duration= 0.054259300231933594
len(imageLists)=32
len(imageLists[1])=40


In [53]:
'''图片转化为视频'''
def pic2vid(imgList,save2=''):
    import cv2
    img_array = []
    for filename in imgList:
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    fps = 20
    out = cv2.VideoWriter(f'/Users/kailong/Downloads/video/{save2}.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, size) 
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
for i in tqdm(range(1,33)):
    pic2vid(imageLists[i],save2=str(i))

100%|██████████| 32/32 [00:21<00:00,  1.46it/s]


In [51]:
len(imageLists[32])

40

In [2]:
def preloadimages(parameterRange=np.arange(1,20),tune=1):
    '''
    purpose:
        preload images into image object sequences corrresponding too each parameter
        each parameter corresponds to 40 image objects
    steps:

    '''
    tune=tune-1
    start = time.time()
    imageLists={}
    numberOfUpdates=8 # corresponds to 66 updates    
    last_image=''
    for currParameter in tqdm(parameterRange): #49
        images=[]
        print('maximum morph=',round((tune*currParameter*numberOfUpdates+2)/numberOfUpdates+1))
        for axis in ['bedTable', 'benchBed']:
            tmp_images=[]
            for currImg in range(1,int(round(tune*currParameter*numberOfUpdates+2)),int((currParameter*numberOfUpdates+2)/numberOfUpdates)):
                currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
                if currMorph<1 or currMorph>99:
                    raise Exception('morphing outside limit')
                curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                if curr_image!=last_image:
                    currImage=visual.ImageStim(win=mywin,
                                                name='image',
                                                image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
                                                ori=0, pos=(0, 0), size=(0.5, 0.5),
                                                color=[1,1,1], colorSpace='rgb', opacity=1,
                                                flipHoriz=False, flipVert=False,
                                                texRes=128, interpolate=True, depth=-4.0)
                tmp_images.append(currImage)
                last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
            images=images+sample(tmp_images)
            tmp_images=[]
            for currImg in reversed(range(1,int(round(tune*currParameter*numberOfUpdates+1)),int((currParameter*numberOfUpdates+2)/numberOfUpdates))):
                currMorph=100-round(currImg/numberOfUpdates+1) if axis=='benchBed' else round(currImg/numberOfUpdates+1)
                curr_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
                if curr_image!=last_image:
                    currImage=visual.ImageStim(win=mywin,
                                                name='image',
                                                image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5), mask=None,
                                                ori=0, pos=(0, 0), size=(0.5, 0.5),
                                                color=[1,1,1], colorSpace='rgb', opacity=1,
                                                flipHoriz=False, flipVert=False,
                                                texRes=128, interpolate=True, depth=-4.0)
                tmp_images.append(currImage)
                last_image=cfg.feedback_expScripts_dir+'carchair_exp_feedback/{}_{}_{}.png'.format(axis,currMorph,5)
            images=images+sample(tmp_images)
        imageLists.update( {currParameter : images} )
    end = time.time()
    print("preload image duration=", end - start)
    return imageLists

_=time.time()
imageLists=preloadimages(parameterRange=parameterRange,tune=tune)
print(f"time passed {(time.time()-_)/60} min")

  0%|          | 0/19 [00:00<?, ?it/s]

maximum morph= 4.0


  5%|▌         | 1/19 [00:00<00:08,  2.06it/s]

maximum morph= 7.0


 11%|█         | 2/19 [00:01<00:10,  1.65it/s]

...

maximum morph= 55.0


 95%|█████████▍| 18/19 [00:58<00:03,  3.83s/it]

maximum morph= 58.0


100%|██████████| 19/19 [01:02<00:00,  3.27s/it]


preload image duration= 62.10655117034912
time passed 1.0390954494476319 min


In [3]:
try:
    for i in range(10**9):
        pass
except:
    print("work")

work


In [3]:
import os
if 'watts' in os.getcwd():
    main_dir = "/home/watts/Desktop/ntblab/kailong/rtSynth_rt/"
else:
    main_dir="/Users/kailong/Desktop/rtEnv/rtSynth_rt/"
import sys
sys.path.append(main_dir)
sys.path.append(main_dir+"expScripts/feedback/")
from psychopy import visual, event, core, logging, gui, data, monitors
from psychopy.hardware.emulator import launchScan, SyncGenerator
from PIL import Image
import string
import fmrisim as sim
import numpy as np
import pandas as pd
import pylink   
from tqdm import tqdm
import time
import re
import logging
import threading
import argparse
alpha = string.ascii_uppercase
from rtCommon.subjectInterface import SubjectInterface
from rtCommon.wsRemoteService import WsRemoteService, parseConnectionArgs
from rtCommon.utils import installLoggers
from rtCommon.cfg_loading import mkdir,cfg_loading
sys.path.append(f'{main_dir}expScripts/recognition/')
from recognition_dataAnalysisFunctions import AdaptiveThreshold



class SubjectService:
    def __init__(self, args, webSocketChannelName='wsSubject'):
        """
        Uses the WsRemoteService framework to parse connection-related args and establish
        a connection to a remote projectServer. Instantiates a local version of
        SubjectInterface to handle client requests coming from the projectServer connection.
        Args:
            args: Argparse args related to connecting to the remote server. These include
                "-s <server>", "-u <username>", "-p <password>", "--test",
                "-i <retry-connection-interval>"
            webSocketChannelName: The websocket url extension used to connecy and communicate
                to the remote projectServer, 'wsSubject' will connect to 'ws://server:port/wsSubject'
        """
        self.subjectInterface = SubjectInterface(subjectRemote=False)
        self.wsRemoteService = WsRemoteService(args, webSocketChannelName)
        self.wsRemoteService.addHandlerClass(SubjectInterface, self.subjectInterface)

    def runDetached(self):
        """Starts the receiver in it's own thread."""
        self.recvThread = threading.Thread(name='recvThread',
                                        target=self.wsRemoteService.runForever)
        self.recvThread.setDaemon(True)
        self.recvThread.start()


argParser = argparse.ArgumentParser()

argParser.add_argument('-c', '--config', action="store", dest="config", default='sub001.ses2.toml', type=str, help='experiment file (.json or .toml)')
argParser.add_argument('-r', '--run', action="store", dest="run", default='1', type=str, help='current run')
# argParser.add_argument('-e', '--sess', action="store", dest="sess", default='1', type=str, help='current session')
argParser.add_argument('-s', action="store", dest="server", default="localhost:7777",
                    help="Server Address with Port [server:port]")

argParser.add_argument('-i', action="store", dest="interval", type=int, default=5,
                    help="Retry connection interval (seconds)")
argParser.add_argument('-u', '--username', action="store", dest="username", default='kp578',
                    help="rtcloud website username")
argParser.add_argument('-p', '--password', action="store", dest="password", default='kp578',
                    help="rtcloud website password")
argParser.add_argument('--test', default=False, action='store_true',
                    help='Use unsecure non-encrypted connection')
argParser.add_argument('--trying', default=True, action='store_true',
                    help='Use unsecure non-encrypted connection')
args = argParser.parse_args("")



if args.trying:
    scanmode = 'Test'  # 'Scan' or 'Test' or None
    screenmode = False  # fullscr True or False
    monitor_name = "testMonitor" #"testMonitor"
else:
    scanmode = 'Scan'  # 'Scan' or 'Test' or None
    screenmode = True  # fullscr True or False
    monitor_name = "scanner"

if not re.match(r'.*:\d+', args.server):
    print("Error: Expecting server address in the form <servername:port>")
    argParser.print_help()
    sys.exit()

# Check if the ssl certificate is valid for this server address
from rtCommon.projectUtils import login, certFile, checkSSLCertAltName, makeSSLCertFile
addr, _ = args.server.split(':')
if checkSSLCertAltName(certFile, addr) is False:
    # Addr not listed in sslCert, recreate ssl Cert
    makeSSLCertFile(addr)


cfg = cfg_loading(args.config)
sub = cfg.subjectName
run = int(args.run)  # 1
cfg.run = run
sess = int(cfg.session)

cfg.feedback_expScripts_dir = f"{cfg.projectDir}expScripts/feedback/"

gui = True if screenmode == False else False
scnWidth, scnHeight = monitors.Monitor(monitor_name).getSizePix()
frameTolerance = 0.001  # how close to onset before 'same' frame
TRduration=int(cfg.TR)

# mywin = visual.Window(
    # size=[1280, 800], fullscr=screenmode, screen=0,
    # winType='pyglet', allowGUI=False, allowStencil=False,
    # monitor=monitor_name, color=[0,0,0], colorSpace='rgb', #color=[0,0,0]
    # blendMode='avg', useFBO=True,
    # units='height')

mywin = visual.Window(
    size=[scnWidth - 100, scnHeight - 100], fullscr=screenmode, screen=1,
    winType='pyglet', allowGUI=False, allowStencil=False,
    monitor=monitor_name, color=[0,0,0], colorSpace='rgb', #color=[0,0,0]
    blendMode='avg', useFBO=True,
    units='height')

# similation specific
step=3 #in simulation, how quickly the morph changes ramp up. Note this is only for simulation, has nothing to do with real experiment

# trial_list designing parameters
parameterRange=[1,5,9,13] #np.arange(1,prange) #for saving time for now. np.arange(1,20) #define the range for possible parameters for preloading images. Preloading images is to make the morphing smooth during feedback
tune=4 # this parameter controls how much to morph (how strong the morphing is) (used in preloading function), tune can range from (1,6.15] when paremeterrange is np.arange(1,20)
TrialNumber=cfg.TrialNumber # how many trials are required #test trial ,each trial is 14s, 10 trials are 140s.

## - design the trial list: the sequence of the different types of components: 
## - e.g: ITI + waiting for fMRI signal + feedback (receive model output from feedbackReceiver.py)
trial_list = pd.DataFrame(columns=['Trial','time','TR','state','newWobble'])
curTime=0
curTR=0
state=''
trial_list.append({'Trial':None,
                    'time':None,
                    'TR':None,
                    'state':None,
                    'newWobble':None},
                    ignore_index=True)

for currTrial in range(1,1+TrialNumber):

    # ITI
    for i in range(6): # should be 6TR=12s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'ITI',
                                    'newWobble':0},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1

    # waiting for metric calculation
    for i in range(3): # should be 3TR=6s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'waiting',
                                    'newWobble':0},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1

    # feedback trial: try minimize the whobbling
    for i in range(5): #5TR=10s
        trial_list=trial_list.append({'Trial':currTrial,
                                    'time':curTime,
                                    'TR':curTR,
                                    'state':'feedback',
                                    'newWobble':1},
                                    ignore_index=True)
        curTime=curTime+TRduration
        curTR=curTR+1

# ITI
for i in range(6): # should be 6TR=12s
    trial_list=trial_list.append({'Trial':currTrial,
                                'time':curTime,
                                'TR':curTR,
                                'state':'ITI',
                                'newWobble':0},
                                ignore_index=True)
    curTime=curTime+TRduration
    curTR=curTR+1
trial_list

conda env=rtcloud
toml filename=/Users/kailong/Desktop/rtEnv/rtSynth_rt/projects/rtSynth_rt/conf/sub001.ses2.toml


,Trial,time,TR,state,newWobble
0,1,0,0,ITI,0
1,1,1,1,ITI,0
2,1,2,2,ITI,0
3,1,3,3,ITI,0
4,1,4,4,ITI,0
...,...,...,...,...,...
169,12,169,169,ITI,0
170,12,170,170,ITI,0
171,12,171,171,ITI,0
172,12,172,172,ITI,0


In [8]:
trial_list.iloc[12:]

,Trial,time,TR,state,newWobble
12,1,12,12,feedback,1
13,1,13,13,feedback,1
14,2,14,14,ITI,0
15,2,15,15,ITI,0
16,2,16,16,ITI,0
...,...,...,...,...,...
169,12,169,169,ITI,0
170,12,170,170,ITI,0
171,12,171,171,ITI,0
172,12,172,172,ITI,0


In [17]:
list(trial_list['state'])

['ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'feedback',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'ITI',
 'waiting',
 'waiting',
 'waiting',
 'feedback',
 'feedbac

In [71]:
import pandas as pd
a=pd.read_csv("/Users/kailong/Desktop/rtEnv/rtSynth_rt/subjects/sub001/adaptiveThreshold.csv")
# a["successful trials"].iloc[-1] = 1
# a = a.append({
#         'sub':1, 
#         'session':1, 
#         'run':1, 
#         'threshold':1},
#         ignore_index=True)
a=a.drop(["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1"   ,"Unnamed: 0.1.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1"],axis=1)
a=a.drop([0,5,7,8,9,10,12,14,16,18,20])
a

,sub,session,run,threshold,successful_trials,perfect_trials
1,sub001,4,1,0.6,7.0,4.0
2,sub001,4,2,0.6,5.0,3.0
3,sub001,4,3,0.6,4.0,8.0
4,sub001,4,4,0.6,8.0,3.0
6,sub001,5,1,0.6,8.0,2.0
11,sub001,5,2,0.6,7.0,4.0
13,sub001,5,3,0.6,5.0,3.0
15,sub001,5,4,0.6,4.0,8.0
17,sub001,5,5,0.6,8.0,3.0
19,sub001,5,6,0.6,5.0,5.0


In [68]:
c=float(1)
c==None


False

In [41]:
states=list(trial_list['state'])
states
def countITI(states):
    c=0
    for i in states:
        if i == "ITI":
            c+=1
        else:
            return c
    return c
        

countITI(states)

6

In [2]:
"OFF"==1

False

In [1]:
import pandas as pd
a=pd.read_csv("/Users/kailong/Desktop/rtEnv/rtSynth_rt/subjects/sub002/ses1/recognition/sub002_1.csv")
a

,Unnamed: 0,Sub,Run,TR,Onset,Item,Change,CorrResp,Resp,RT,Acc,image_on,button_on,button_off
0,0,sub002,1,6,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,sub002,1,7,2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,sub002,1,8,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,sub002,1,9,6,D,0.0,NaN,NaN,NaN,NaN,18.010208,19.027508,19.927112
4,4,sub002,1,10,8,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,sub002,1,97,182,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,92,sub002,1,98,184,D,0.0,NaN,NaN,NaN,NaN,195.998464,197.011361,197.912250
93,93,sub002,1,99,186,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,94,sub002,1,100,188,C,0.0,NaN,NaN,NaN,NaN,199.999492,201.014199,201.912885
